In [1]:
import time
start_time = time.time()
import pandas as pd
import sys
import numpy as np

In [2]:
loja = 1
periodo = 2

In [3]:
#Please store RAW FILE and GROUPO-OUTROS file in the file folder path
file_folder_path = f"C:\\Bots\\Bot_transmissao\\Geracao CAT\\Loja_{loja}\\"
raw_file_name = f"loja_{loja}_tabela_2_p{periodo}_v1.csv"
raw_file_path = file_folder_path + raw_file_name

cnpj_file_name = "C:\\Bots\\Bot_transmissao\\Geracao CAT\\Referencias\\cnpjs.xlsx"
cnpj_file_path = cnpj_file_name
missing_cnpj_file_name=  f"C:\\Bots\\Bot_transmissao\\Geracao CAT\\Loja_{loja}\\missing_cnpj_loja_{loja}.xlsx"
missing_cnpj_file_path = missing_cnpj_file_name

final_file_name = "final.csv"
final_file_path = file_folder_path + final_file_name

In [4]:
#Import raw file , groupo
df= pd.read_csv(raw_file_path, dtype={"CODIGO_BARRA": str,"Valor Base Cálculo ICMS Substituição Tributária":str, "NUM_ITEM":str, "CFOP":str,"COD_ITEM":str}, encoding='utf-8')

#Please make sure the cnpj column is text
cnpj= pd.read_excel(cnpj_file_path, dtype={"cnpj": str})

In [5]:
df

,Unnamed: 0,CHV_DOC,DATA,CFOP,NUM_ITEM,COD_ITEM,IND_OPER,SUB_TIPO,QTD_CAT,QTD_EFD,...,UNIDADE,N C M,CEST,CNPJ EMITENTE,CNPJ DESTINATARIO,Valor Base Cálculo ICMS Substituição Tributária,vBCST,FONTE,COMBINACAO_CHV_NUM,CHAVE_ITEM
0,0,35191003129126000159550020000027291157936550,2019-10-01,1411,1,94771,0,-1,1,NaN,...,UN,30049079.0,1300200.0,3.129126e+12,0.000000e+00,NaN,0.0,entrada_oprprp,35191003129126000159550020000027291157936550-1,35191003129126000159550020000027291157936550-1
1,1,35191003129126000159550020000027301158021920,2019-10-02,1411,1,148098,0,-1,1,NaN,...,UN,82142000.0,2802600.0,3.129126e+12,0.000000e+00,NaN,0.0,entrada_oprprp,35191003129126000159550020000027301158021920-1,35191003129126000159550020000027301158021920-1
2,2,35191003129126000159550020000027311158028019,2019-10-02,1411,1,155283,0,-1,1,NaN,...,UN,33049910.0,2800800.0,3.129126e+12,0.000000e+00,NaN,0.0,entrada_oprprp,35191003129126000159550020000027311158028019-1,35191003129126000159550020000027311158028019-1
3,3,35191003129126000159550020000027311158028019,2019-10-02,1411,2,155283,0,-1,1,NaN,...,UN,33049910.0,2800800.0,3.129126e+12,0.000000e+00,NaN,0.0,entrada_oprprp,35191003129126000159550020000027311158028019-2,35191003129126000159550020000027311158028019-2
4,4,35191003129126000159550020000027321158122292,2019-10-03,1411,1,78550,0,-1,1,NaN,...,UN,30049029.0,1300200.0,3.129126e+12,0.000000e+00,NaN,0.0,entrada_oprprp,35191003129126000159550020000027321158122292-1,35191003129126000159550020000027321158122292-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
493477,560479,35200503129126000159550020000031271171064100,2020-05-09,5411,1,211409,1,-1,1,NaN,...,UN,30049099.0,1300301.0,3.129126e+12,2.784996e+13,NaN,0.0,saida_nfe,35200503129126000159550020000031271171064100-1,35200503129126000159550020000031271171064100-1
493478,560572,35200703129126000159550020000032161174195310,2020-07-04,5411,1,224790,1,-1,3,NaN,...,UN,33049990.0,2001500.0,3.129126e+12,2.784996e+13,NaN,0.0,saida_nfe,35200703129126000159550020000032161174195310-1,35200703129126000159550020000032161174195310-1
493479,560573,35200703129126000159550020000032171174195334,2020-07-04,5411,1,224790,1,-1,2,NaN,...,UN,33049990.0,2001500.0,3.129126e+12,2.784996e+13,NaN,0.0,saida_nfe,35200703129126000159550020000032171174195334-1,35200703129126000159550020000032171174195334-1
493480,560719,35201003129126000159550020000033711179926296,2020-10-08,5411,1,44831,1,-1,175,NaN,...,UN,30066000.0,1300500.0,3.129126e+12,5.651966e+12,NaN,0.0,saida_nfe,35201003129126000159550020000033711179926296-1,35201003129126000159550020000033711179926296-1


In [6]:
## remove duplicate in cnpj just in case
cnpj.drop_duplicates(subset='cnpj', inplace=True)

In [7]:
## Add chave-item
df['chave-item'] = df['CHV_DOC'] +'-'+ df['NUM_ITEM']

In [8]:
## Add cnpj
df['cnpj'] = df['CHV_DOC'].apply(lambda x: x[6:20])

### DFA1

In [9]:
dfa1= df.loc[df['CFOP'].str[0].isin(['1','2']),['CHV_DOC', 'DATA', 'CFOP', 'NUM_ITEM', 'COD_ITEM','IND_OPER', 'SUB_TIPO', 'QTD_CAT','cnpj','chave-item']]

In [10]:
## Merge with cpnj
dfa1= dfa1.merge(cnpj, left_on='cnpj', right_on='cnpj', how='left')

In [11]:
dfa1['grupo/outros'] = np.where(dfa1['CFOP']=='1411', 'grupo', dfa1['grupo/outros'])

In [12]:
## Checking missing cnpj, stop if there are missing cnpj in groupo and export missing cnpj to "missing cnpj.xlsx" file
if dfa1.loc[dfa1['grupo/outros'].isna()].shape[0] > 0:
    dfa1.loc[dfa1['grupo/outros'].isna(),'cnpj'].drop_duplicates().to_excel(missing_cnpj_file_path,index=False)
    print("Please update ""groupo.csv"" file with missing cnpj in the ""missing cnpj.xlsx"" ")
    sys.exit()
else:
    pass

In [13]:
dfa1.rename(columns={"grupo/outros":"tipo"}, inplace=True)

In [14]:
dfa1['cod-tipo'] = dfa1['COD_ITEM'] + "-" + dfa1['tipo']

### DFA2

In [15]:
#Take the unique cod-tipo
dfa2 = dfa1[['COD_ITEM','cod-tipo','tipo']].drop_duplicates().sort_values(['cod-tipo','tipo'])

In [16]:
#Extract the maximum cod item
max_cod_item = df.loc[np.where(df['COD_ITEM'].str.isnumeric(), True, False ), 'COD_ITEM'].astype(float).astype(int).max() +1

In [17]:
max_cod_item

901002574

In [18]:
#new cod-item
dfa2['tipo transform'] = dfa2.groupby('COD_ITEM')['cod-tipo'].transform('cumcount').cumsum().shift(1,fill_value=0) + max_cod_item

In [19]:
#no longer need after adjustments
# dfa2['cod-tipo novo'] = np.where(dfa2.groupby('COD_ITEM')['cod-tipo'].transform('cumcount') >0, dfa2['tipo transform'].astype(str) +'-'+ dfa2['tipo'], dfa2['cod-tipo']   )

In [20]:
dfa2['tipo2'] = np.where(dfa2.groupby('COD_ITEM')['cod-tipo'].transform('cumcount') >0 , dfa2['tipo'] + str(2), dfa2['tipo'] )
dfa2['cod2'] = np.where(dfa2.groupby('COD_ITEM')['cod-tipo'].transform('cumcount') >0 , dfa2['tipo transform'].astype(str), dfa2['COD_ITEM'] )
dfa2['cod2-tipo2'] = np.where(dfa2.groupby('COD_ITEM')['cod-tipo'].transform('cumcount') >0 , dfa2['cod2'].astype(str) + '-' + dfa2['tipo2'] , dfa2['cod-tipo'] )
dfa2['novo'] = np.where(dfa2.groupby('COD_ITEM')['cod-tipo'].transform('cumcount') >0, True, False)

### DFA5

In [21]:
dfa5 = dfa1.merge(dfa2[['cod-tipo', 'cod2-tipo2' , 'cod2', 'tipo2', 'novo']],left_on='cod-tipo', right_on='cod-tipo', how='left') \
        .groupby(['COD_ITEM', 'tipo', 'cod2-tipo2','cod2', 'tipo2', 'novo'], as_index=False, observed=True)['QTD_CAT'].sum() \
        .rename( columns={'QTD_CAT' : 'Total por forn'})

In [22]:
#no longer need after adjustments
# dfa5['total'] = dfa5.groupby('COD_ITEM')['Total por forn'].transform('sum')
# dfa5['fracao'] = np.where( dfa5['novo'], dfa5['Total por forn'] / dfa5['total'], 0)

### DFA6

In [23]:
#Only cod-novo not blank
dfa6 = dfa5.loc[dfa5['novo'],['COD_ITEM', 'cod2']].drop_duplicates(subset='cod2')
# dfa6[dfa6['cod novo'].duplicated()]

## DFB1

In [24]:
dfb1 = df.loc[df['CFOP'].str[0].isin(['5','6']),['CHV_DOC', 'DATA', 'CFOP', 'NUM_ITEM', 'COD_ITEM','IND_OPER', 'SUB_TIPO', 'QTD_CAT','chave-item']]

In [25]:
#no longer need after adjustments
#SUM QTD_CAT over COD_ITEM
# dfb1['qtd tot saida'] = dfb1.groupby('COD_ITEM')['QTD_CAT'].transform('sum')

In [26]:
#ADD qtd total entrada
dfb1 = dfb1.merge( dfa5.loc[~dfa5['novo']].groupby(['COD_ITEM'],as_index=False, observed=True)['Total por forn'].sum()\
                .rename(columns={'Total por forn':'qtd total entrada'}),\
                left_on='COD_ITEM', right_on='COD_ITEM', how= 'left')
dfb1['qtd total entrada'].fillna(0,inplace=True)

In [27]:
dfb1 = dfb1.sort_values(['COD_ITEM'])
dfb1['qtd acum saida'] = dfb1.groupby('COD_ITEM')['QTD_CAT'].transform('cumsum')

In [28]:
dfb1['qtd total need'] = dfb1['qtd total entrada'] - dfb1['qtd acum saida']

In [29]:
dfb1 = dfb1.merge(dfa6, left_on='COD_ITEM', right_on='COD_ITEM',how='left').rename(columns={'cod2':'cod2 test'})
dfb1['cod2'] = np.where(dfb1['qtd total need']<0, dfb1['cod2 test'].fillna(dfb1['COD_ITEM']) , dfb1['COD_ITEM'] )

In [30]:
#no longer need after adjustments
# dfb1['qtd acum saida pct'] =  dfb1['qtd acum saida'] / dfb1['qtd tot saida'] 
# dfb1['cod novo'] = np.where( (~dfb1['cod novo analise'].isna()) & (~dfb1['pct entrada'].isna()) & (dfb1['pct entrada'] > dfb1['qtd acum saida pct'] ) , dfb1['cod novo analise'] , dfb1['COD_ITEM'] )

In [31]:
dfb1 = dfb1.merge(dfa2[['cod2', 'tipo2']].drop_duplicates(), left_on='cod2', right_on='cod2', how='left')

### DFC1

In [32]:
dfc1 =pd.concat([ dfb1[['chave-item', 'tipo2', 'cod2']] , \
                 dfa1.merge(dfa2[['cod-tipo','cod2-tipo2','cod2','tipo2']],left_on='cod-tipo', right_on='cod-tipo', how='left')[['chave-item','tipo2','cod2']] ] ).drop_duplicates(subset='chave-item')

In [33]:
df_final = df.merge(dfc1, left_on= 'chave-item' , right_on ='chave-item', how='left')[[ \
        'CHV_DOC', 'DATA', 'CFOP', 'NUM_ITEM', 'cod2','tipo2', 'COD_ITEM',
       'IND_OPER', 'SUB_TIPO', 'QTD_CAT', 'QTD_EFD', 'ICMS_TOT', 'VL_CONFR_0',
       'COD_LEGAL', 'ALIQUOTA', 'VALOR',
       'Valor Base Cálculo ICMS ST Retido Operação Anterior',
       'Valor Complementar', 'Valor ICMS Substituição Tributária',
       'Valor ICMS Operação', 'DESCRICAO', 'CODIGO_BARRA', 'UNIDADE', 'N C M',
       'CEST', 'CNPJ EMITENTE', 'CNPJ DESTINATARIO',
       'Valor Base Cálculo ICMS Substituição Tributária', 'vBCST', 'FONTE']]

df_final.rename(columns={'cod2': 'cod novo',
                        'tipo2': 'tipo'}, inplace = True)
df_final

,CHV_DOC,DATA,CFOP,NUM_ITEM,cod novo,tipo,COD_ITEM,IND_OPER,SUB_TIPO,QTD_CAT,...,DESCRICAO,CODIGO_BARRA,UNIDADE,N C M,CEST,CNPJ EMITENTE,CNPJ DESTINATARIO,Valor Base Cálculo ICMS Substituição Tributária,vBCST,FONTE
0,35191003129126000159550020000027291157936550,2019-10-01,1411,1,94771,grupo,94771,0,-1,1,...,Nimesulida Novaq 100mg 12S,SEM GTIN,UN,30049079.0,1300200.0,3.129126e+12,0.000000e+00,NaN,0.0,entrada_oprprp
1,35191003129126000159550020000027301158021920,2019-10-02,1411,1,148098,grupo,148098,0,-1,1,...,Kit Concare Manicure,SEM GTIN,UN,82142000.0,2802600.0,3.129126e+12,0.000000e+00,NaN,0.0,entrada_oprprp
2,35191003129126000159550020000027311158028019,2019-10-02,1411,1,155283,grupo,155283,0,-1,1,...,Creme Prev Assadur 45g Le Blue,SEM GTIN,UN,33049910.0,2800800.0,3.129126e+12,0.000000e+00,NaN,0.0,entrada_oprprp
3,35191003129126000159550020000027311158028019,2019-10-02,1411,2,155283,grupo,155283,0,-1,1,...,Creme Prev Assadur 45g Le Blue,SEM GTIN,UN,33049910.0,2800800.0,3.129126e+12,0.000000e+00,NaN,0.0,entrada_oprprp
4,35191003129126000159550020000027321158122292,2019-10-03,1411,1,78550,grupo,78550,0,-1,1,...,Ibuprofeno Prati 600mg 20S,SEM GTIN,UN,30049029.0,1300200.0,3.129126e+12,0.000000e+00,NaN,0.0,entrada_oprprp
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
493477,35200503129126000159550020000031271171064100,2020-05-09,5411,1,211409,grupo,211409,1,-1,1,...,DULCOGAS 125MG CAP GEL 10 S,7891058022495,UN,30049099.0,1300301.0,3.129126e+12,2.784996e+13,NaN,0.0,saida_nfe
493478,35200703129126000159550020000032161174195310,2020-07-04,5411,1,224790,grupo,224790,1,-1,3,...,AGUA MICELAR LE BLUE 150ML,7898623355309,UN,33049990.0,2001500.0,3.129126e+12,2.784996e+13,NaN,0.0,saida_nfe
493479,35200703129126000159550020000032171174195334,2020-07-04,5411,1,224790,grupo,224790,1,-1,2,...,AGUA MICELAR LE BLUE 150ML,7898623355309,UN,33049990.0,2001500.0,3.129126e+12,2.784996e+13,NaN,0.0,saida_nfe
493480,35201003129126000159550020000033711179926296,2020-10-08,5411,1,44831,NaN,44831,1,-1,175,...,ALDIJET AMP 1ML,7899420500732,UN,30066000.0,1300500.0,3.129126e+12,5.651966e+12,NaN,0.0,saida_nfe


In [34]:
df_final['Valor Complementar'] = np.where(df_final['tipo'].isin(['outros', 'outros2']) ,
                                         0,
                                         df_final['Valor Complementar'])

df_final['vBCST'] = df_final['vBCST'].fillna(0)
df_final['ICMS_TOT'] = np.where((df_final['FONTE'] == 'entrada_sp') | (df_final['FONTE'] == 'entrada_outras'),
    np.where(df_final['Valor Base Cálculo ICMS ST Retido Operação Anterior'] <= df_final['vBCST'],
             np.maximum(df_final['Valor ICMS Operação'].fillna(0) + df_final['Valor ICMS Substituição Tributária'].fillna(0), (df_final['Valor Base Cálculo ICMS ST Retido Operação Anterior'] + df_final['Valor Complementar'].fillna(0)) * df_final['ALIQUOTA'].fillna(0).astype(float)/100),
             np.maximum(df_final['Valor ICMS Operação'].fillna(0) + df_final['Valor ICMS Substituição Tributária'].fillna(0), (df_final['vBCST'].astype(float).fillna(0) + df_final['Valor Complementar'].fillna(0)) * df_final['ALIQUOTA'].fillna(0).astype(float)/100)),
                         np.nan)

In [35]:
df_final.to_csv(f'C:\\Bots\\Bot_transmissao\\Geracao CAT\\Loja_{loja}\\loja_{loja}_tabela_2_1_p{periodo}_v1.csv')

In [36]:
print("--- %s seconds ---" % (time.time() - start_time))

--- 11.18134093284607 seconds ---


In [37]:
df_final['ICMS_TOT_UNIT'] = df_final['ICMS_TOT']/df_final['QTD_CAT']

df_final[['ICMS_TOT_UNIT']].describe()

,ICMS_TOT_UNIT
count,143455.000000
mean,2.723255
std,3.686610
min,0.000000
25%,0.715200
50%,1.721820
75%,3.520000
max,265.401000
